In [1]:
from __future__ import print_function, division, absolute_import

import argparse
import time

import numpy as np
seed = 7
np.random.seed(seed)

import os
import sys
sys.path.insert(0, os.path.abspath('..'))
os.chdir('../CoMET')

import evolutron.networks as nets
from evolutron.motifs import motif_extraction
from evolutron.tools import load_dataset, none2str, Handle
from evolutron.engine import DeepTrainer

Using Theano backend.
Using gpu device 1: GeForce GTX 1080 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


In [2]:
# Data and model
data_id = 'zinc'
padded = True
mode = 'unsuper'
model = None

# Architecture
n_filters = 100
filter_length = 10
n_convs = 3
n_fc = 1

# Training
num_epochs = 20
batch_size = 50
optimizer = 'nadam'
rate = .1
validate = .2

In [3]:
dataset = load_dataset(data_id, padded=padded)

Dataset size: 11906


In [4]:
x_data = dataset
# Find input shape
if type(x_data) == np.ndarray:
    input_shape = x_data[0].shape
elif type(x_data) == list:
    input_shape = (None, x_data[0].shape[1])
else:
    raise TypeError('Something went wrong with the dataset type')

In [5]:
print('Building model ...')
net_arch = nets.DeepCoDER.from_options(input_shape, n_conv_layers=n_convs, n_fc_layers=n_fc,
                                       n_filters=n_filters, filter_length=filter_length)
conv_net = DeepTrainer(net_arch)
conv_net.compile(optimizer=optimizer, lr=rate)

Building model ...


In [ ]:
conv_net.display_network_info()

Neural Network has 230720 trainable parameters
  #  Name            Shape       Parameters
---  --------------  --------  ------------
  0  aa_seq          7182x20              0
  1  masking_1       7182x20              0
  2  Conv1           7182x100         20100
  3  Conv2           7182x100        100100
  4  Conv3           7182x100        100100
  5  maxpooling1d_1  1x100                0
  6  flatten_1       100                  0
  7  FCEnc1          100              10100
  8  FCDec1          100                100
  9  reshape_1       1x100                0
 10  Unpooling       7182x100             0
 11  Deconv3         7182x100           100
 12  Deconv2         7182x100           100
 13  Deconv1         7182x20             20


In [ ]:
print('Started training at {}'.format(time.asctime()))
conv_net.fit(x_data[:int(.9 * len(x_data))], x_data[:int(.9 * len(x_data))],
             nb_epoch=num_epochs,
             batch_size=batch_size,
             validate=validate)

Started training at Fri Nov 11 22:17:04 2016
Train on 8572 samples, validate on 2143 samples
Epoch 1/20
5950/8572 [===================>..........] - ETA: 29s - loss: 0.0560 - mean_cat_acc: 0.9242

In [ ]:
score = conv_net.score(x_data[int(.9 * len(x_data)):], x_data[int(.9 * len(x_data)):])
print('Test Loss:{0:.6f}, Test Accuracy: {1:.2f}%'.format(score[0], 100*score[1]))

In [ ]:
handle = Handle(batch_size=batch_size,
        filter_size=filter_length,
        filters=n_filters,
        program='CoMET',
        dataset=data_id,
        extra=None)
handle.model = 'realDeepCoDER'
conv_net.save_train_history(handle)
conv_net.save_model_to_file(handle)